In [ ]:
import math
import collections

import numpy as np
import pandas as pd
import matplotlib.pyplot as pp

%matplotlib inline

In [ ]:
#from google.colab import drive#to open the google drive in colab
#drive.mount('/content/drive')


In [ ]:
#read all parameters of header
timeSteps=[]
boxDim=[]
num_frame=0

with open(r'C:\vmd\nvt') as fp:
#with open('/content/drive/My Drive/Case/nanoBloodProject/MSD/disc/12rc/170lig/bare/rc1.0/nvt') as fp:

        for line in fp:
            if 'ITEM: TIMESTEP' in line:                                         
                for line in fp: # now you are at the lines you want
                    if 'ITEM: NUMBER OF ATOMS' in line: 
                     break 
                   
                    line=line.rstrip()# to remove any white spaces at the end of the string                  
                    #
                    timeSteps.append(line)# add the string in to the list
                    num_frame+=1
                    

            if 'ITEM: NUMBER OF ATOMS' in line:                                         
                for line in fp: # now you are at the lines you want
                    if 'ITEM: BOX BOUNDS' in line: 
                     break 
                   
                    line=line.rstrip()# to remove any white spaces at the end of the string                  
                    #
                    num_atom=int(line)  

            if 'ITEM: BOX BOUNDS' in line:                                         
                for line in fp: # now you are at the lines you want
                    if 'ITEM: ATOMS' in line: 
                     break 
                   
                    line=line.rstrip()# to remove any white spaces at the end of the string                  
                    
                    boxDim.append(line)               
                    

In [ ]:
#read coordinate part
time_counter=0
lines=[]

#df.loc[:,'x'] = df.loc[:,'x'].astype(float)
with open(r'C:\vmd\nvt') as fp:
#with open('/content/drive/My Drive/Case/nanoBloodProject/MSD/rod/8rc/76lig/homo/corNpAij/nvt') as fp:  
       
       for line in fp:
            atom_counter=0
            
            if 'ITEM: ATOMS' in line:
                time_counter+=1                                         
                
                for line in fp: # now you are at the lines you want
                    atom_counter+=1

                    #x(time_counter,atom_counter)=df.loc[line,'x'] 
                                        
                    if 'ITEM: TIMESTEP' in line: 
                     break 
                    
                    
                    line=line.rstrip()# to remove any white spaces at the end of the string                  
                    lines.append(line)# add the string in to the list
                    #lines[time_counter][atom_counter]=line# add the string in to the list
                   
                    

In [ ]:

num_frame
#num_atom
#boxDim #i have to modify the code to get x max x min and so on for box dimensionpr


In [ ]:
#remove empty lines from the list
while("" in lines) : 
    lines.remove("") 

In [ ]:
#data frame for the whole dump file without separating by frame
df=pd.DataFrame(lines, columns=['atomID']) # to make a data frame of pandas from the list and put atomID name on the column
df[['atomID','atomType','x','y','z']] = df.atomID.str.split(expand=True,) # to split one column to multiple columns and allocate name for each column
#df

In [ ]:
df.iloc[1]

**lines of data frame is read for each frame**

**read the center of mass of NP**

#  very important: to read all lines in the entire dump file( all the lines of all the frames)

In [ ]:
type_Frame=np.zeros((num_frame,num_atom))


In [ ]:
for i in range (0, num_frame):
   
   type_Frame[i,:]=df.atomType.values[(i*num_atom):((i*num_atom)+num_atom)] #thankGOD! to read x values for each frame and put in an array

In [ ]:
for i in range (0, num_frame):
  num_atom_type=0
  for k in range (0, num_atom):
     if type_Frame[i,k]==5: #we only read atoms of core of NP with atom type 5
       num_atom_type+=1

In [ ]:
num_atom_type

In [ ]:
ID_X_Frame=np.zeros ((num_frame+1,num_atom_type+1))
ID_Y_Frame=np.zeros ((num_frame+1,num_atom_type+1))
ID_Z_Frame=np.zeros ((num_frame+1,num_atom_type+1))

In [ ]:
time_counter=0
for i in range (0, num_frame):
  atom_counter=0
  time_counter+=1
  for k in range (0, num_atom):
     if type_Frame[i,k]==5:
       atom_counter+=1
       
 #print(time_counter,"",atom_counter)
       ID_X_Frame[time_counter,atom_counter]=df.x.values[k]

should we consider image flages as we did for catenane in MD???
 here in dump of DPD I did not save image flages!

In [ ]:
#calculate center of mass 

mass=1 #each bead mass

Xcm=np.zeros(num_frame+1)
Ycm=np.zeros(num_frame+1)
Zcm=np.zeros(num_frame+1)

total_mass= num_atom_type * mass

time_counter=0
for i in range (0, num_frame):
  atom_counter=0
  time_counter+=1
  Xoverall=0 # after each frame this will be zero so count from begining for next frame
  Yoverall=0
  Zoverall=0
  for k in range (0, num_atom):
     
     if type_Frame[i,k]==5: #core beads of NP
       atom_counter+=1
  
       ID_X_Frame[time_counter,atom_counter]=df.x.values[k]
       ID_Y_Frame[time_counter,atom_counter]=df.y.values[k]
       ID_Z_Frame[time_counter,atom_counter]=df.z.values[k]

       Xoverall = Xoverall + ID_X_Frame[time_counter,atom_counter]
       Yoverall = Yoverall + ID_Y_Frame[time_counter,atom_counter]
       Zoverall = Zoverall + ID_Z_Frame[time_counter,atom_counter]

       Xcm[time_counter] = Xoverall/total_mass
       Ycm[time_counter] = Yoverall/total_mass
       Zcm[time_counter] = Zoverall/total_mass     
       


In [ ]:
for i in range (0, num_frame):
  print(i,'',Zcm[i]) #here the center of box is in wall so Np is located above EG with distsnce around 14.9 nm so Zcm starts from 14.9 and when np go inside EG and be closer to wall the Zcm also decreases.
#Xoverall

**calculate MSD**

In [ ]:
#non time average msd of center of mass of NP
Xmsd=np.zeros (num_frame)# make 1D arrays
Ymsd=np.zeros (num_frame)
Zmsd=np.zeros (num_frame)
Rmsd=np.zeros (num_frame)

for i in range (1, 2):#thank god I should use frame one bc frame zero is zero but in future I shoul fix the frame zero so can have all frames info
 
 for t in range (1, num_frame):  

        
         
        dx=Xcm[t]-Xcm[i]
        dy=Ycm[t]-Ycm[i]
        dz=Zcm[t]-Zcm[i]       
        
        Xmsd[t]+=np.power(dx,2)
        Ymsd[t]+=np.power(dy,2)
        Zmsd[t]+=np.power(dz,2)
        
        Rmsd[t]= Xmsd[t]+Ymsd[t]+ Zmsd[t];
       
        print(t,'',dz,'',Zmsd[t],'',Rmsd[t])
        #print(t,'',Xmsd[t])

**non time ave MSD**

In [ ]:
#thankGOD! non time ave MSD:
for t in range (1,num_frame):
  print(t,'',Rmsd[t])

In [ ]:
df = pd.DataFrame(Rmsd)
df.to_excel(r'C:\vmd\RMSDnonTimAve.xlsx')

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import pylab 

df =pd.ExcelFile(r'C:\vmd\RMSDnonTimAve.xlsx')# the name of execl file should start with capital word. Rdf is correct but not rdf 
Sheet1 =pd.read_excel(df,header=0)

ax=Sheet1.plot(kind='line', color='blue', linewidth=0.5, label='MSD')
ax.legend()
#Sheet1.plot(kind='line',  x='a', y= 'c', color='red', linewidth=0.5, label='toluene', ax=ax)
#Sheet1.plot(kind='line',  x='a', y= 'd', color='green', linewidth=0.5, label='interface', ax=ax)
ax.legend()
ax.set_xlabel('time (100ps)')
ax.set_ylabel('Distance (0.71nm)') 
pylab.xlim([0,30])
pylab.ylim([0,150])# define the range of y coordinate
plt.ticklabel_format(axis="both", style="sci", scilimits=(0,0))#show the x and y axis in scientific mode
plt.savefig(r'C:\vmd\RMSDnonTimAve.png')

**Zmsd only the motion in z direction**

In [ ]:
df = pd.DataFrame(Zmsd)
df.to_excel(r'C:\vmd\ZMSDnonTimAve.xlsx')

In [ ]:
#logaritmic scale plot

import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import pylab 

df =pd.ExcelFile(r'C:\vmd\ZMSDnonTimAve.xlsx')# the name of execl file should start with capital word. Rdf is correct but not rdf 
Sheet1 =pd.read_excel(df,header=0)

ax=Sheet1.plot(kind='line', color='blue', linewidth=0.5, label='Z coordinate')
ax.legend()
#Sheet1.plot(kind='line',  x='a', y= 'c', color='red', linewidth=0.5, label='homo', ax=ax)
#Sheet1.plot(kind='line',  x='a', y= 'd', color='green', linewidth=0.5, label='janus', ax=ax)
#Sheet1.plot(kind='line',  x='a', y= 'e', color='orange', linewidth=0.5, label='random', ax=ax)
#Sheet1.plot(kind='line',  x='a', y= 'f', color='blue', linewidth=0.5, label='janusSecPos', ax=ax)
ax.legend()
ax.set_xlabel('time (100ps)')
ax.set_ylabel('Distance (0.71nm)') 
#ax.set_yscale('log') #to plot in log scale

pylab.xlim([0,30])
pylab.ylim([0,100])# define the range of y coordinate
#plt.ticklabel_format(axis="x", style="sci", scilimits=(0,0))#show only the x axis in scientific mode when Y coordinate is in log scale
plt.ticklabel_format(axis="both", style="sci", scilimits=(0,0))#show the x and y axis in scientific mode
plt.savefig(r'C:\vmd\ZMSDnonTimAve.png')

In [ ]:
#logaritmic scale plot

import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import pylab 

df =pd.ExcelFile(r'C:\vmd\Zmsd_8r,12,16rcDisc.xlsx')# the name of execl file should start with capital word. Rdf is correct but not rdf 
Sheet1 =pd.read_excel(df,header=0)

ax=Sheet1.plot(kind='line', y= '8rc', color='blue', linewidth=0.5, label='8rc')
ax.legend()
Sheet1.plot(kind='line',  y= '12rc', color='red', linewidth=0.5, label='12rc', ax=ax)
Sheet1.plot(kind='line',  y= '16rc', color='green', linewidth=0.5, label='16rc', ax=ax)
#Sheet1.plot(kind='line',  y= '(-)perpend', color='orange', linewidth=0.5, label='(-)perpend', ax=ax)
#Sheet1.plot(kind='line',  y= 'parallel', color='black', linewidth=0.5, label='parallel', ax=ax)

ax.legend()
ax.set_xlabel('time (100ps)')
ax.set_ylabel('Distance (0.71nm)') 
#ax.set_yscale('log') #to plot in log scale

pylab.xlim([0,30])
pylab.ylim([0,100])# define the range of y coordinate
#plt.ticklabel_format(axis="x", style="sci", scilimits=(0,0))#show only the x axis in scientific mode when Y coordinate is in log scale
plt.ticklabel_format(axis="both", style="sci", scilimits=(0,0))#show the x and y axis in scientific mode
plt.savefig(r'C:\vmd\Zmsd_8r,12,16rcDisc.png')

In [ ]:
#logaritmic scale plot

import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import pylab 

df =pd.ExcelFile(r'C:\vmd\z_msd_perpend_parallel.xlsx')# the name of execl file should start with capital word. Rdf is correct but not rdf 
Sheet1 =pd.read_excel(df,header=0)

ax=Sheet1.plot(kind='line', y= 'perpend', color='red', linewidth=0.5, label='perpendicular')
ax.legend()
Sheet1.plot(kind='line',  y= 'parallel', color='green', linewidth=0.5, label='parallel', ax=ax)
#Sheet1.plot(kind='line',  y= '16rc', color='green', linewidth=0.5, label='16rc', ax=ax)
#Sheet1.plot(kind='line',  y= '(-)perpend', color='orange', linewidth=0.5, label='(-)perpend', ax=ax)
#Sheet1.plot(kind='line',  y= 'parallel', color='black', linewidth=0.5, label='parallel', ax=ax)

ax.legend()
ax.set_xlabel('time (100ps)')
ax.set_ylabel('Distance (1nm)') 
#ax.set_yscale('log') #to plot in log scale

pylab.xlim([0,30])
pylab.ylim([0,100])# define the range of y coordinate
#plt.ticklabel_format(axis="x", style="sci", scilimits=(0,0))#show only the x axis in scientific mode when Y coordinate is in log scale
plt.ticklabel_format(axis="both", style="sci", scilimits=(0,0))#show the x and y axis in scientific mode
plt.savefig(r'C:\vmd\z_msd_perpend_parallel.png')

In [ ]:
#logaritmic scale plot

import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import pylab 

df =pd.ExcelFile(r'C:\vmd\rc1.0_ZMSD_12rc_rod_disc_sphere.xlsx')# the name of execl file should start with capital word. Rdf is correct but not rdf 
Sheet1 =pd.read_excel(df,header=0)

ax=Sheet1.plot(kind='line', y= 'rod', color='blue', linewidth=0.5, label='rod')
ax.legend()
Sheet1.plot(kind='line',  y= 'disc', color='red', linewidth=0.5, label='disc', ax=ax)
Sheet1.plot(kind='line',  y= 'sphere', color='green', linewidth=0.5, label='sphere', ax=ax)
#Sheet1.plot(kind='line',  y= '(-)perpend', color='orange', linewidth=0.5, label='(-)perpend', ax=ax)
#Sheet1.plot(kind='line',  y= 'parallel', color='black', linewidth=0.5, label='parallel', ax=ax)

ax.legend()
ax.set_xlabel('time (100ps)')
ax.set_ylabel('Distance (0.71nm)') 
ax.set_yscale('log') #to plot in log scale

pylab.xlim([0,30])
pylab.ylim([0,100])# define the range of y coordinate
plt.ticklabel_format(axis="x", style="sci", scilimits=(0,0))#show only the x axis in scientific mode when Y coordinate is in log scale
#plt.ticklabel_format(axis="both", style="sci", scilimits=(0,0))#show the x and y axis in scientific mode
plt.savefig(r'C:\vmd\rc1.0_ZMSD_12rc_rod_disc_sphere(log).png')

# correct plot units

In [ ]:
#ThankGOD correct plot units

import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import pylab 
from matplotlib import rcParams
import matplotlib

df =pd.ExcelFile(r'C:\vmd\z_msd_bare_170,340lig.xlsx')# the name of execl file should start with capital word. Rdf is correct but not rdf 
Sheet1 =pd.read_excel(df,header=0)

ax=Sheet1.plot(kind='line', y= 'bare', color='blue', linewidth=1.5, label='bare')
ax.legend()
Sheet1.plot(kind='line',  y= '170 lig', color='red', linewidth=1.5, label='170 lig', ax=ax)
Sheet1.plot(kind='line',  y= '340 lig', color='green', linewidth=1.5, label='340 lig', ax=ax)
#Sheet1.plot(kind='line',  y= '(-)perpend', color='orange', linewidth=0.5, label='(-)perpend', ax=ax)
#Sheet1.plot(kind='line',  y= 'parallel', color='black', linewidth=0.5, label='parallel', ax=ax)

ax.legend()

##########################################
ax.set_xlabel('time (100ps)')
ax.set_ylabel('distance (0.1nm)') 

pylab.xlim([1,30])
plt.xticks(np.arange(10,31,10)) #the distance of units to be shown in x axis. it starts from 10 to 30 with idistance of 10.thankGOD!
pylab.ylim([0,100])# define the range of y coordinate
plt.yticks(np.arange(0,100.1,20))

rcParams['font.family'] = 'serif'
rcParams['font.sans-serif'] = ['Bitstream Vera Sans']
rcParams['font.serif'] = ['Bitstream Vera Sans']
rcParams["font.size"] = "17"
matplotlib.rcParams['legend.fontsize'] = 10

plt.rcParams["figure.figsize"] = [5.50, 4.0]
plt.rcParams["figure.autolayout"] = True

plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))#show the x and y axis in scientific 
###########################################


plt.savefig(r'C:\vmd\z_msd_bare_170,340lig2.png')

In [ ]:
#thankGOD! final plot mehtod

import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import pylab 
from matplotlib import rcParams
import matplotlib

df =pd.ExcelFile(r'C:\vmd\Z_msd_perpend_parallel.xlsx')# the name of execl file should start with capital word. Rdf is correct but not rdf 
Sheet1 =pd.read_excel(df,header=0)

ax=Sheet1.plot(kind='line', y= 'perpend', color='red', linewidth=1.5, label='perpendicular')
ax.legend()
#Sheet1.plot(kind='line', y= '45degree', color='red', linewidth=1.5, label='45˚', ax=ax)
#Sheet1.plot(kind='line', y= '(-)45degree', color='green', linewidth=1.5, label='135˚', ax=ax)
#Sheet1.plot(kind='line', y= '(-)perpend', color='orange', linewidth=1.5, label='180˚', ax=ax)
Sheet1.plot(kind='line', y= 'parallel', color='green', linewidth=1.5, label='parallel', ax=ax)
ax.legend()


##########################################
ax.set_xlabel('time (100ps)')
ax.set_ylabel('distance (0.1nm)') 

pylab.xlim([1,30])
plt.xticks(np.arange(0,31,10)) #the distance of units to be shown in x axis. it starts from 10 to 30 with idistance of 10.thankGOD!
pylab.ylim([0,100])# define the range of y coordinate
plt.yticks(np.arange(0,100.1,20))

rcParams['font.family'] = 'serif'
rcParams['font.sans-serif'] = ['Bitstream Vera Sans']
rcParams['font.serif'] = ['Bitstream Vera Sans']
rcParams["font.size"] = "17"
matplotlib.rcParams['legend.fontsize'] = 10

#plt.rcParams["figure.figsize"] = [5.50, 4.0]
plt.rcParams["figure.autolayout"] = True

plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))#show the x and y axis in scientific 
###########################################


plt.savefig(r'C:\vmd\Z_msd_perpend_parallel2.png')

In [ ]:
#ThankGOD correct plot units
#thankGOD! 

import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import pylab 
from matplotlib import rcParams
import matplotlib

df =pd.ExcelFile(r'C:\vmd\z_msd_rod_disc_sphere.xlsx')# the name of execl file should start with capital word. Rdf is correct but not rdf 
Sheet1 =pd.read_excel(df,header=0)

ax=Sheet1.plot(kind='line', y= 'rod', color='blue', linewidth=1.5, label='rod')
ax.legend()
Sheet1.plot(kind='line',  y= 'disc', color='red', linewidth=1.5, label='disc', ax=ax)
Sheet1.plot(kind='line',  y= 'sphere', color='green', linewidth=1.5, label='sphere', ax=ax)
#Sheet1.plot(kind='line',  y= '(-)perpend', color='orange', linewidth=0.5, label='(-)perpend', ax=ax)
#Sheet1.plot(kind='line',  y= 'parallel', color='black', linewidth=0.5, label='parallel', ax=ax)
ax.legend()


##########################################
ax.set_xlabel('time (100ps)')
ax.set_ylabel('distance (0.1nm)') 

pylab.xlim([1,30])
plt.xticks(np.arange(0,31,10)) #the distance of units to be shown in x axis. it starts from 10 to 30 with idistance of 10.thankGOD!
pylab.ylim([0,100])# define the range of y coordinate
plt.yticks(np.arange(0,100.1,20))

rcParams['font.family'] = 'serif'
rcParams['font.sans-serif'] = ['Bitstream Vera Sans']
rcParams['font.serif'] = ['Bitstream Vera Sans']
rcParams["font.size"] = "17"
matplotlib.rcParams['legend.fontsize'] = 10

plt.rcParams["figure.figsize"] = [5.50, 4.0]
plt.rcParams["figure.autolayout"] = True

plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))#show the x and y axis in scientific 
###########################################


plt.savefig(r'C:\vmd\z_msd_rod_disc_sphere2.png')

In [ ]:
#logaritmic scale plot

import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import pylab 
from matplotlib import rcParams
import matplotlib

df =pd.ExcelFile(r'C:\vmd\Zmsd_8r,12,16rcDisc.xlsx')# the name of execl file should start with capital word. Rdf is correct but not rdf 
Sheet1 =pd.read_excel(df,header=0)

ax=Sheet1.plot(kind='line', y= '8rc', color='blue', linewidth=1.5, label='8rc')
ax.legend()
Sheet1.plot(kind='line',  y= '12rc', color='red', linewidth=1.5, label='12rc', ax=ax)
Sheet1.plot(kind='line',  y= '16rc', color='green', linewidth=1.5, label='16rc', ax=ax)
#Sheet1.plot(kind='line',  y= '(-)perpend', color='orange', linewidth=0.5, label='(-)perpend', ax=ax)
#Sheet1.plot(kind='line',  y= 'parallel', color='black', linewidth=0.5, label='parallel', ax=ax)
ax.legend()


##########################################
ax.set_xlabel('time (100ps)')
ax.set_ylabel('distance (0.1nm)') 

pylab.xlim([1,30])
plt.xticks(np.arange(10,31,10)) #the distance of units to be shown in x axis. it starts from 10 to 30 with idistance of 10.thankGOD!
pylab.ylim([0,100])# define the range of y coordinate
plt.yticks(np.arange(0,100.1,20))

rcParams['font.family'] = 'serif'
rcParams['font.sans-serif'] = ['Bitstream Vera Sans']
rcParams['font.serif'] = ['Bitstream Vera Sans']
rcParams["font.size"] = "17"
matplotlib.rcParams['legend.fontsize'] = 10

plt.rcParams["figure.figsize"] = [5.50, 4.0]
plt.rcParams["figure.autolayout"] = True

plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))#show the x and y axis in scientific 
###########################################


plt.savefig(r'C:\vmd\Zmsd_8r,12,16rcDisc.png')

# Xmsd only the motion in x direction


In [ ]:
df = pd.DataFrame(Xmsd)
df.to_excel(r'C:\vmd\XMSDnonTimAve.xlsx')

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import pylab 

df =pd.ExcelFile(r'C:\vmd\XMSDnonTimAve.xlsx')# the name of execl file should start with capital word. Rdf is correct but not rdf 
Sheet1 =pd.read_excel(df,header=0)

ax=Sheet1.plot(kind='line', y= 'b', color='blue', linewidth=0.5, label='X coordinate')
ax.legend()
#Sheet1.plot(kind='line',  x='a', y= 'c', color='red', linewidth=0.5, label='toluene', ax=ax)
#Sheet1.plot(kind='line',  x='a', y= 'd', color='green', linewidth=0.5, label='interface', ax=ax)
ax.legend()
ax.set_xlabel('time (100ps)')
ax.set_ylabel('Distance (0.71nm)') 
pylab.xlim([0,30])
pylab.ylim([0,2])# define the range of y coordinate
plt.ticklabel_format(axis="both", style="sci", scilimits=(0,0))#show the x and y axis in scientific mode
plt.savefig(r'C:\vmd\XMSDnonTimAve.png')

# Ymsd only the motion in y direction


In [ ]:
df = pd.DataFrame(Ymsd)
df.to_excel(r'C:\vmd\YMSDnonTimAve.xlsx')

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import pylab 

df =pd.ExcelFile(r'C:\vmd\YMSDnonTimAve.xlsx')# the name of execl file should start with capital word. Rdf is correct but not rdf 
Sheet1 =pd.read_excel(df,header=0)

ax=Sheet1.plot(kind='line', y= 'b', color='blue', linewidth=0.5, label='Y coordinate')
ax.legend()
#Sheet1.plot(kind='line',  x='a', y= 'c', color='red', linewidth=0.5, label='toluene', ax=ax)
#Sheet1.plot(kind='line',  x='a', y= 'd', color='green', linewidth=0.5, label='interface', ax=ax)
ax.legend()
ax.set_xlabel('time (100ps)')
ax.set_ylabel('Distance (0.71nm)') 
pylab.xlim([0,30])
pylab.ylim([0,70])# define the range of y coordinate
plt.ticklabel_format(axis="both", style="sci", scilimits=(0,0))#show the x and y axis in scientific mode
plt.savefig(r'C:\vmd\YMSDnonTimAve.png')

# **time ave MSD**

In [ ]:
#thankGOd! time ave MSD:
MSD=np.zeros  (num_frame)

for i in range (0,num_frame):
  if i==0:
    Rmsd[i]=0
    
  MSD[i]=Rmsd[i]/(num_frame)
  print(i,"",MSD[i])

In [ ]:
df = pd.DataFrame(MSD)
df.to_excel(r'C:\vmd\MSD_TimAve.xlsx')

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import pylab 

df =pd.ExcelFile(r'C:\vmd\MSD_TimAve.xlsx')# the name of execl file should start with capital word. Rdf is correct but not rdf 
Sheet1 =pd.read_excel(df,header=0)

ax=Sheet1.plot(kind='line', y= 'b', color='blue', linewidth=0.5, label='Y coordinate')
ax.legend()
#Sheet1.plot(kind='line',  x='a', y= 'c', color='red', linewidth=0.5, label='toluene', ax=ax)
#Sheet1.plot(kind='line',  x='a', y= 'd', color='green', linewidth=0.5, label='interface', ax=ax)
ax.legend()
ax.set_xlabel('time (100ps)')
ax.set_ylabel('Distance (0.71nm)') 
pylab.xlim([0,30])
pylab.ylim([0,10])# define the range of y coordinate
plt.ticklabel_format(axis="both", style="sci", scilimits=(0,0))#show the x and y axis in scientific mode
plt.savefig(r'C:\vmd\MSD_TimAve.png')

In [ ]:
MSDx=np.zeros (num_frame)

for i in range (0,num_frame):
 if i==0:
    Xmsd[i]=0
    
 MSDx[i]=Xmsd[i]/((num_frame-i)*num_atom_type)
 print(MSDx)

In [ ]:
MSDy=np.zeros (num_frame)

for i in range (0,num_frame):
 if i==0:
    Ymsd[i]=0
    
 MSDy[i]=Ymsd[i]/((num_frame-i)*num_atom_type)
 print(MSDy)

In [ ]:
#non time ave MSD of NP that from far distance reach to wall (at zero Z)
MSDz=np.zeros (num_frame)

for i in range (1,num_frame):
 if i==0:
    Zmsd[i]=0
    
 MSDz[i]=Zmsd[i]/(num_frame)
 print(i,"",MSDz[i])